# Project 4: Finetuning and Prompting

In this project, you will first learn how to use Huggingface's Transformers library to load large language models. Next, we will generate text from these models. Finally, we will work with a small text-to-SQL dataset, where the input is a natural language query and the output is a SQL query that can be executed against a database. You will explore (1) finetuning a pretrained language model, and (2) prompting the pretrained language model with examples. 

This project will be more open ended than the previous projects. We expect you to learn how to use the huggingface and torch documentation.

## Setup

First we install and import the required dependencies. These include:
* `torch` for modeling and training
* `transformers` for pre-trained models
* `datasets` from huggingface to load existing datasets.

In [99]:
%%capture
!pip install transformers
!pip install datasets
!pip install sentence-transformers

# Standard library imports
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Before proceeding, let's verify that we're connected to a GPU runtime and that `torch` can detect the GPU.
We'll define a variable `device` here to use throughout the code so that we can easily change to run on CPU for debugging.

In [100]:
assert torch.cuda.is_available()
device = torch.device("cuda")
print("Using device:", device)

Using device: cuda


## Loading Model

We will use GPT-2 Medium for this project. This includes both the GPT-2 tokenizer and the GPT-2 model weights itself. If you want to learn more about this model, you can read the GPT-2 paper https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf.

Let's first load the tokenizer for the GPT-2 medium model. You can find how to do this by reading the documentation for AutoTokenzier in transformers, and finding the GPT-2 model of ~345 million params in there.

In [101]:
from transformers import AutoTokenizer
# Your code here
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")

tokenizer.pad_token = tokenizer.eos_token # convenient for padding later

Let's tokenize and detokenize some text from this model.

In [102]:
print(tokenizer.encode('Hello world'))
print(tokenizer.decode(tokenizer.encode('Hello world')))
print(tokenizer.encode("Hola, cómo estás😍"))

[15496, 995]
Hello world
[39, 5708, 11, 269, 10205, 5908, 1556, 40138, 47249, 235]


Now let's load the GPT-2 Medium model. Make sure you also put the model onto the GPU.

In [103]:
from transformers import AutoModelForCausalLM
# Your code here
gpt2_model = AutoModelForCausalLM.from_pretrained('gpt2-medium').to(device)

## Generate From the Model

Now let's generate some text from the model to test its LM capabilities. Let's first generate one output of length 50 tokens using greedy decoding (temperature = 0), which should get us some text with high likelihood under the model. When generating text, you can condition on phrases such as "The coolest thing in NLP right now is". Find the relevant function and arguments to use for generating text using the Huggingface documentation.

Hint: you may find https://huggingface.co/docs/transformers/main_classes/text_generation to be useful for learning about generating from LMs.

In [104]:
inputs = tokenizer("The coolest thing right now in NLP is", return_tensors="pt").input_ids.cuda()
# Your code here
sample_output = gpt2_model.generate(inputs, max_length=50, temperature=0, num_return_sequences=1)[0]
print(sample_output)
print("{}".format(tokenizer.decode(sample_output, skip_special_tokens=True)))

tensor([  464, 38889,  1517,   826,   783,   287,   399, 19930,   318,   262,
         2694,   284,   779,   262,   976,  1366,   284,  4331,   262,  2003,
           13,   198,   198,   464,  2003,   318,  1464,  5609,    11,   290,
          262,  1366,   318,  1464,  5609,    13,   198,   198,   464,  2003,
          318,  1464,  5609,    11,   290,   262,  1366,   318,  1464,  5609],
       device='cuda:0')
The coolest thing right now in NLP is the ability to use the same data to predict the future.

The future is always changing, and the data is always changing.

The future is always changing, and the data is always changing


Now let's generate 10 pieces of random text of length 50 tokens from the model using random sampling with temperature set to 0.7. This will allow the text to be somewhat higher in diversity (random sampling) while maintaining reasonable quality (temperature < 1). 

In [105]:
inputs = tokenizer("The coolest thing right now in NLP is", return_tensors="pt").input_ids.cuda()
# Your code here
# sample_outputs =[gpt2_model.generate(inputs, max_length=50, temperature=0.7, num_return_sequences=1,do_sample=True)[0] for _ in range(10)] 
sample_outputs =gpt2_model.generate(inputs, max_length=50, temperature=0.7, num_return_sequences=10,do_sample=True) 

for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

0: The coolest thing right now in NLP is the way it's using deep learning and machine learning to make sense of the data," he said.

And when you can do that with a computer, that's when you start to see the application
1: The coolest thing right now in NLP is the open-source tool that lets you do more than just see things. We've found a great community of people who love to do cool stuff with their NLP tools. So we're adding another community
2: The coolest thing right now in NLP is that it's not just about what we're talking about, it's about what you're talking about.

I see that a lot.

It's not about what I'm talking about.
3: The coolest thing right now in NLP is the use of the "Literal" class. This means that the description of the data is not just a string, but a literal representation of the data. This concept is very powerful, and is
4: The coolest thing right now in NLP is that it can be used both as a training tool and as a learning tool.

It's really fun to observe and to 

## Text-to-SQL Task Setup

First, let's download the data of text-to-SQL pairs and the database against which we'll execute queries to retrieve answers.

The code below initializes the database and does some initial preprocessing data preprocessing + splitting for you.

In [106]:
%%capture
!wget https://github.com/jkkummerfeld/text2sql-data/raw/master/data/geography.json
!wget https://github.com/jkkummerfeld/text2sql-data/raw/master/data/geography-db.sql

import re
import sqlite3
import json
from copy import deepcopy

DATABASE_NAME = 'geo.db'
SQL_FILENAME = 'geography-db.sql'
DATASET_FILENAME = 'geography.json'

with open(SQL_FILENAME, 'r') as file:
    sql_script = file.read()
    sql_script = re.sub(r"\s*ENGINE=[^ ]+","", sql_script)
    sql_script = re.sub(r"\s*DEFAULT CHARSET=[^ ;]+","", sql_script)
    sql_script = re.sub(r"\s*LOCK TABLES `[^`]+` WRITE;","", sql_script)  # remove LOCK TABLES
    sql_script = re.sub(r"\s*UNLOCK TABLES;","", sql_script)  # remove UNLOCK TABLES
    sql_script = sql_script.replace('`', '')  # remove backticks

# Connect to the SQLite database (this will create the file if it doesn't exist)
connection = sqlite3.connect(DATABASE_NAME)
print(sql_script)

connection.executescript(sql_script)
connection.commit()
connection.close()

connection = sqlite3.connect(DATABASE_NAME)
cursor = connection.cursor()
with open(DATASET_FILENAME, 'r') as file:
    dataset = json.load(file)

splits = {'train': [], 'dev': [], 'test': []}
for query_type in dataset:
    for example in query_type['sentences']:
        split = example['question-split']
        example['question'] = example['text']
        for key, value in example['variables'].items():
            example['question'] = example['question'].replace(key, value)
        example['sql'] = deepcopy(query_type['sql'])
        example['sql'] = example['sql'][0]
        for key, value in example['variables'].items():
            example['sql'] = example['sql'].replace(key, value)
        try:
            cursor.execute(example['sql'])
        except:
            continue
        example['db_answer'] = cursor.fetchall()
        del example['text']
        del example['variables']
        splits[split].append(example)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


We also provide a function you can use to query the database:

In [107]:
def query_db(sql):
    connection = sqlite3.connect(DATABASE_NAME)
    cursor = connection.cursor()
    cursor.execute(sql)
    result = cursor.fetchall()
    connection.close()
    return result

This dataset is pretty small:

In [108]:
print('Train set size:', len(splits['train']))
print('Dev set size:', len(splits['dev']))
print('Test set size:', len(splits['test']))

Train set size: 547
Dev set size: 48
Test set size: 277


Let's inspect an example from the training dataset:

In [109]:
splits['train'][0]

{'question-split': 'train',
 'question': 'what is the biggest city in nebraska',
 'sql': 'SELECT CITYalias0.CITY_NAME FROM CITY AS CITYalias0 WHERE CITYalias0.POPULATION = ( SELECT MAX( CITYalias1.POPULATION ) FROM CITY AS CITYalias1 WHERE CITYalias1.STATE_NAME = "nebraska" ) AND CITYalias0.STATE_NAME = "nebraska" ;',
 'db_answer': [('omaha',)]}

Note that the `db_answer` is the result of executing the given SQL output against the database:

In [110]:
query_db(splits['train'][0]['sql'])

[('omaha',)]

Let's check how well our language model does on this text-to-SQL task out of the box. You can just use greedy decoding. 

In [111]:
prompt = "Write a SQL query based on the following question.\n\nQuestion: {input}\n\nSQL:"

In [112]:
# Your code here. Generate from the model using greedy decoding with the above prompt
inputs = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
output = gpt2_model.generate(inputs, max_length=50, temperature=0, num_return_sequences=1)[0]
predicted_sql = tokenizer.decode(output, skip_special_tokens=True)

print(predicted_sql)

Write a SQL query based on the following question.

Question: {input}

SQL: SELECT * FROM {input}

Output:

SELECT * FROM {input}

Question: {input}

SQL:


In [113]:
# query_db("SELECT * FROM city WHERE city_id = 1;")

You should get something that looks kind of like a SQL query, but it probably won't match the correct output, and in fact it most likely won't even execute without crashing when you try to query the database (you'll see a syntax error below).

In [114]:
try:
    query_db(predicted_sql)
    print('success!')
except:
    print('failed to execute!')

failed to execute!


Let's confirm quantitatively that the model doesn't work well out-of-the-box by running on the dev dataset (`splits['dev']`).

In [128]:
@torch.no_grad()
def predict_greedy(model, data, max_new_tokens=128):
    """
    Return the model's greedy text-to-sql predictions on the given data split.
    The maximum number of new tokens generated (NOT including tokens in the prompt) should be equal to max_new_tokens.
    For speed, you should batch the generation. The tokenizer can handle multiple inputs simultaneously,
    but you'll need to tell it to pad using padding=True, and you may also need to set tokenizer.padding_side='left'.
    Hint: as a postprocessing step after you're done, you may need to cut off the output at the first appearance of '\n' if the output is continuing past the end of the SQL.
    """
    questions = [d['question'] for d in data]
    predicted_sqls = []
    # Your code here
    batch_size=8
    model=model.to(device)
    model.eval()
    tokenizer.padding_side='left'
#     print(questions)
    PROMPT = "Question: {question}\n\nSQL:"
    questions=[PROMPT.format(question=i) for i in questions]

    for i in range(0, len(questions), batch_size):
        batch_questions = questions[i:i + batch_size]
        inputs = tokenizer(batch_questions, return_tensors="pt", padding=True).input_ids.cuda()
        output = model.generate(inputs, max_length=max_new_tokens, num_return_sequences=1, temperature=0,do_sample=False)
        generated_sqls = tokenizer.batch_decode(output, skip_special_tokens=True, max_length=max_new_tokens)
        
        for i in range(len(generated_sqls)):
#             generated_sqls[i]=generated_sqls[i].replace("SQL:","")
#             print(i, generated_sqls[i].split('\n\n'))
            if '\n\n' in generated_sqls[i]:
                components = generated_sqls[i].split('\n\n')
#                 print(components)
                predicted_sqls.append(components[1].replace("SQL: ", ""))
#                 print(predicted_sqls)

#     predicted_sqls = [sql.replace("SQL:", "").strip() for sql in all_sqls]
#     predicted_sqls=all_sqls
    return predicted_sqls # list of strings containing SQL predictions for each question in the data

In [ ]:
prompt="Write a SQL query based on the following question.\n\nQuestion: {what is the biggest city in arizona?}\n\nSQL:"
inputs = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
output = gpt2_model.generate(inputs, max_length=500, temperature=0, num_return_sequences=1)[0]
predicted_sql = tokenizer.decode(output, skip_special_tokens=True)

print(predicted_sql)

In [182]:
finetuned_cnt=0
def check_execution_accuracy(predictions, data):
    assert len(predictions) == len(data)
    correct = 0
    for p, d in zip(predictions, data):
        try:
#             print(query_db(p),d['db_answer'])
            if query_db(p) == d['db_answer']:
                correct += 1
        except: # failed to execute
#             print("??!!?")
            global finetuned_cnt
            finetuned_cnt+=1
            pass
    return correct / len(predictions)

In [ ]:
# print(predictions)

In [129]:
predictions = predict_greedy(gpt2_model, splits['dev'])
# print(predictions)
print('example prediction:', predictions[0])
print('initial execution acc', check_execution_accuracy(predictions, splits['dev']))

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


example prediction: SELECT CITYalias0.CITY_NAME FROM CITY AS CITYalias0 WHERE CITYalias0.POPULATION = ( SELECT MAX( CITYalias1.POPULATION ) FROM CITY AS CITYalias1 WHERE CITYalias1.STATE_NAME = "arizona" ) AND CITYalias0.STATE_NAME = "arizona" ;
initial execution acc 0.1875


You will probably observe an accuracy around 0-2%. (It may be hard to verify if your `predict_greedy` function is correct at this stage, because the expected accuracy is so low, but you will reuse it later with an improved model, at which point it will be more obvious if your implementation is correct.)

## Model Finetuning

Now let's prepare our dataset for finetuning (i.e., training our pretrained language model on this text-to-SQL training set). For each element in the dataset, it should have a text prompt and then the SQL output, similar to above. Your job is to fill in the labels field below. This field sets the labels to use for training during the language modeling task.

For the labels, we only want to train the model to output the text after the word "SQL:". This is because in the prompt, everything before the word "SQL:" will also be provided to the model as input. Hint: use -100 as the label for tokens you do not want to train on. Hint 2: When doing LM training, the labels are the same as the input tokens, except shifted to the left by one. You should check whether Huggingface is already doing the shifting, or whether you need to do the shifting yourself.

One thing to be careful of with all LMs is to make sure there are not extra spaces. So, the text should be formatted as like "SQL: {sql output}" not "SQL: {sql output} ". 

In [130]:
class Text2SQLDataset(Dataset):
    PROMPT = "Write a SQL query based on the following question.\n\nQuestion: {question}\n\nSQL: {sql}"

    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
        tokenizer.padding_side = 'right'

        self.input_ids = []
        self.attn_masks = []
        self.labels = []

        training_texts = []
        for example in self.data:
            training_text = Text2SQLDataset.PROMPT.format(question=example['question'], sql=example['sql']) + "<|endoftext|>" # include the end token so model knows when to stop!
            training_texts.append(training_text)
        encodings_dict = self.tokenizer(training_texts, padding=True, truncation=True)
#         print(encodings_dict.keys())
        for i, (example, training_text) in enumerate(zip(data, training_texts)):
            self.input_ids.append(torch.tensor(encodings_dict['input_ids'][i]))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask'][i]))
            # Your code here
            input_tokens = self.tokenizer.tokenize(training_text)
            sql_start = input_tokens.index('SQL') 
            label = encodings_dict['input_ids'][i]
            label[:sql_start]=[-100]*sql_start
#             print(label)
#             labels = labels[:len(self.input_ids[-1])]  # Ensure label length matches input
            self.labels.append(torch.tensor(label))
        

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {'input_ids':(self.input_ids[idx]),
                'attn_masks': (self.attn_masks[idx]),
                'labels': (self.labels[idx])}
#         return self.input_ids[idx], self.attn_masks[idx], torch.tensor(self.labels[idx])


In [131]:
train_dataset = Text2SQLDataset(splits['train'], tokenizer)
dev_dataset = Text2SQLDataset(splits['dev'], tokenizer)
test_dataset = Text2SQLDataset(splits['test'], tokenizer)

In [122]:
print(len(train_dataset[0]['input_ids']),len(train_dataset[0]['attn_masks']),len(train_dataset[0]['labels']))

302 302 302


Now we can use the Huggingface Trainer to finetune GPT-2 Medium on this dataset. This abstracts away all of the details of training. Setup the training arguments to perform 3 epochs of training on this dataset, use a per-device batch size of 2 with gradient accumulation set to 8, use 30 warmup steps, a weight decay of 0.05. Set the eval batch size to be 8. Save a checkpoint after 100 steps. Set fp16 to True. Save the checkpoint in a specific output_dir so you can load it later. Hint: if it tries to launch Wandb, you may add the argument report_to="none".

In [133]:
from transformers import TrainingArguments, DataCollatorWithPadding

# Define the output directory to save checkpoints
output_dir = "/kaggle/working/checkpoint"


# Training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,  
    num_train_epochs=3,  
    per_device_train_batch_size=2,  
    gradient_accumulation_steps=8,
    warmup_steps=30, 
    weight_decay=0.05, 
    per_device_eval_batch_size=8,
    save_steps=100, 
    evaluation_strategy="steps",  
    eval_steps=100,  
    save_total_limit=2,  
    report_to="none", 
    fp16=True,   
    load_best_model_at_end=True,  
    logging_steps=100
)

# print(train_dataset.__getitem__(0))
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

trainer=Trainer(
    model=gpt2_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    eval_dataset=dev_dataset
)

trainer.train()
# model.save_pretrained(output_dir)

Step,Training Loss,Validation Loss
100,0.023100,0.017629


TrainOutput(global_step=102, training_loss=0.02298957080233331, metrics={'train_runtime': 264.758, 'train_samples_per_second': 6.198, 'train_steps_per_second': 0.385, 'total_flos': 892894929469440.0, 'train_loss': 0.02298957080233331, 'epoch': 2.98})

Reload the final saved version of the model below. You may need to delete the previously loaded model if you run out of GPU memory.

In [134]:
del gpt2_model
# Your code here

finetuned_model = AutoModelForCausalLM.from_pretrained('/kaggle/working/checkpoint/checkpoint-100')

In [162]:
del trainer
import gc
gc.collect()
torch.cuda.empty_cache()

Let's check our finetuned model's performance.

In [137]:
finetuned_predictions = predict_greedy(finetuned_model, splits['test'])
print('finetuned execution acc:', check_execution_accuracy(finetuned_predictions, splits['test']))

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


finetuned execution acc: 0.48736462093862815


You should achieve an accuracy of roughly 50% using the suggested training hyperparameters; we will check >40% in the autograder.

Save your predictions.

In [139]:
def save_predictions(predictions, filename):
    with open(filename, 'w') as f:
        f.write('\n'.join(predictions))

save_predictions(finetuned_predictions, 'finetuned_predictions.txt')

Inspect some of your predictions compared to the correct outputs, and describe some common types of errors in your report. What fraction of errors are due to failing to execute (e.g., syntax error), and what fraction are due to executing but getting the wrong answer?

In [140]:
# Your code here
len(finetuned_predictions),len(splits['test'])
cnt=0
for i in range(len(finetuned_predictions)):
    if finetuned_predictions[i]!=splits['test'][i]['sql']:
        cnt+=1
        print(i,finetuned_predictions[i],"\n",splits['test'][i]['sql'])
print(cnt)

4 SELECT STATEalias0.POPULATION FROM STATE AS STATEalias0 WHERE STATEalias0.STATE_NAME = "new mexico" ; 
 SELECT CITYalias0.CITY_NAME FROM CITY AS CITYalias0 WHERE CITYalias0.POPULATION = ( SELECT MAX( CITYalias1.POPULATION ) FROM CITY AS CITYalias1 WHERE CITYalias1.STATE_NAME = "new mexico" ) AND CITYalias0.STATE_NAME = "new mexico" ;
23 SELECT CITYalias0.POPULATION FROM CITY AS CITYalias0 WHERE CITYalias0.CITY_NAME = "washington" ; 
 SELECT STATEalias0.POPULATION FROM STATE AS STATEalias0 WHERE STATEalias0.STATE_NAME = "washington" ;
24 SELECT STATEalias0.STATE_NAME FROM STATE AS STATEalias0 WHERE STATEalias0.AREA = ( SELECT MIN( STATEalias1.AREA ) FROM STATE AS STATEalias1 ) ; 
 SELECT STATEalias0.STATE_NAME FROM STATE AS STATEalias0 WHERE STATEalias0.POPULATION = ( SELECT MIN( STATEalias1.POPULATION ) FROM STATE AS STATEalias1 ) ;
26 SELECT BORDER_INFOalias0.BORDER FROM BORDER_INFO AS BORDER_INFOalias0 WHERE BORDER_INFOalias0.STATE_NAME IN ( SELECT RIVERalias0.TRAVERSE FROM RIVER A

In [141]:
for i in range(len(finetuned_predictions)):
    if finetuned_predictions[i]==splits['test'][i]['sql']:
        print(i,finetuned_predictions[i])

0 SELECT CITYalias0.CITY_NAME FROM CITY AS CITYalias0 WHERE CITYalias0.POPULATION = ( SELECT MAX( CITYalias1.POPULATION ) FROM CITY AS CITYalias1 WHERE CITYalias1.STATE_NAME = "kansas" ) AND CITYalias0.STATE_NAME = "kansas" ;
1 SELECT CITYalias0.CITY_NAME FROM CITY AS CITYalias0 WHERE CITYalias0.POPULATION = ( SELECT MAX( CITYalias1.POPULATION ) FROM CITY AS CITYalias1 WHERE CITYalias1.STATE_NAME = "louisiana" ) AND CITYalias0.STATE_NAME = "louisiana" ;
2 SELECT CITYalias0.CITY_NAME FROM CITY AS CITYalias0 WHERE CITYalias0.POPULATION = ( SELECT MAX( CITYalias1.POPULATION ) FROM CITY AS CITYalias1 WHERE CITYalias1.STATE_NAME = "california" ) AND CITYalias0.STATE_NAME = "california" ;
3 SELECT CITYalias0.CITY_NAME FROM CITY AS CITYalias0 WHERE CITYalias0.POPULATION = ( SELECT MAX( CITYalias1.POPULATION ) FROM CITY AS CITYalias1 WHERE CITYalias1.STATE_NAME = "rhode island" ) AND CITYalias0.STATE_NAME = "rhode island" ;
5 SELECT CITYalias0.CITY_NAME FROM CITY AS CITYalias0 WHERE CITYalias0

We can also examine the exact match accuracy (i.e., requiring the predicted SQL string to exactly match the gold answer) rather than the execution accuracy (just checking whether the output of executing the SQL against the database is the same).

In [142]:
def check_exact_match_accuracy(predictions, data):
    assert len(predictions) == len(data)
    correct = 0
    for p, d in zip(predictions, data):
        if p == d['sql']:
            correct += 1
    return correct / len(predictions)

In [143]:
print('finetuned exact match acc:', check_exact_match_accuracy(finetuned_predictions, splits['test']))

finetuned exact match acc: 0.4584837545126354


The exact match accuracy will likely be close to the execution accuracy, but not exactly the same. What are some potential pros and cons of each metric? Discuss in your report.

Unload your finetuned model so that you don't run out of GPU memory later.

In [148]:
del finetuned_model

NameError: name 'finetuned_model' is not defined

## Few-Shot Prompting

For the final part of this project, you will explore few-shot prompting, i.e., simply prompting the pretrained language model out-of-the-box using a small number of examples rather than finetuning.

First, let's try just selecting 4 examples completely at random from the training set. Rewrite your `predict_greedy` function to change the prompt:

In [173]:
import random


few_shot_prompt = "Question: {question0}\n\nSQL: {sql0}\n\n\n\n" + \
    "Question: {question1}\n\nSQL: {sql1}\n\n\n\n" + \
    "Question: {question2}\n\nSQL: {sql2}\n\n\n\n" + \
    "Question: {question3}\n\nSQL: {sql3}\n\n\n\n" + \
    "Question: {question}\n\nSQL:"


def select_random_examples(question, few_shot_data, num_examples=4):
    """
    Return a list containing 4 of the elements of few_shot_data, selected randomly
    """
    return random.sample(few_shot_data, num_examples)


@torch.no_grad()
def predict_greedy_fewshot(model, data, few_shot_data, max_new_tokens=512, example_selection_method=select_random_examples):
    """
    Return the model's greedy text-to-sql predictions on the given data split.
    The maximum number of new tokens generated (NOT including tokens in the prompt) should be equal to max_new_tokens.
    The four examples with their SQL outputs should go in {question1}, {sql1}, {question2}, {sql2}, etc. in the few_shot_prompt. 
    The final {question} is the question that we're currently evaluating on.
    """
    questions = [d['question'] for d in data]
    predicted_sqls = []
    prompts = []
    for question in questions:
        few_shot_examples = example_selection_method(question, few_shot_data, num_examples=4)
        prompts.append(few_shot_prompt.format(
            question0=few_shot_examples[0]['question'],
            sql0=few_shot_examples[0]['sql'],
            question1=few_shot_examples[1]['question'],
            sql1=few_shot_examples[1]['sql'],
            question2=few_shot_examples[2]['question'],
            sql2=few_shot_examples[2]['sql'],
            question3=few_shot_examples[3]['question'],
            sql3=few_shot_examples[3]['sql'],
            question=question
        ))
    # Your code here; should be fairly similar to your previous predict_greedy code.
    # Hint: if you batch, we recommend batch size 8-16. 
    batch_size=8
#     print(len(prompts),prompts[0])
    tokenizer.padding_side='left'
    all_sqls=[]
    for i in range(0, len(data)):
#         batch_prompts = prompts[i:i + batch_size]
        inputs = tokenizer(prompts[i], return_tensors="pt", padding=True).input_ids.cuda()
#         print(batch_prompts)
#         print(prompts[i])
        output = model.generate(inputs, max_length=max_new_tokens, num_return_sequences=1, temperature=0,do_sample=False)
        generated_sqls = tokenizer.batch_decode(output, skip_special_tokens=True, max_length=max_new_tokens)
#         print(generated_sqls)
        generated_sqls_string=generated_sqls[0].split("\n")
        sql_strings= [s for s in generated_sqls_string if s.startswith('SQL: ')]
#         print(sql_strings)
        all_sqls.append(sql_strings[4])
#         print(all_sqls)
#         print(sql_strings[4])


    predicted_sqls = [sql.replace("SQL:", "") for sql in all_sqls]
    return predicted_sqls # list of strings containing SQL predictions for each question in the data

In [169]:
# Reload the gpt2 model if you need to
gpt2_model = AutoModelForCausalLM.from_pretrained('gpt2-medium').to(device) # Your code here

In [ ]:
# print(len(predictions),len(splits['test']))
# print(predictions)

In [174]:
# This call can take a few minutes even if you batch; you can debug on a subset of the dev set as needed.
predictions = predict_greedy_fewshot(gpt2_model, splits['test'], splits['train'])
# print(len(predictions),len(splits['test']))

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 600, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 568, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


In [175]:
print('4-shot prompting with random examples, execution acc:', check_execution_accuracy(predictions, splits['test']))

4-shot prompting with random examples, execution acc: 0.06498194945848375


You will probably observe between 0-5% accuracy. Random example selection doesn't work very well on this dataset. 

However, what if we select examples by picking the examples from the training set whose questions are most similar to our current question? To do this, load a pretrained sentence encoder, which takes a sentence as input and outputs a fixed-length vector encoding semantic information about that sentence. First compute the vectors associated with all the training set questions, and then select examples from the training set based on which question vectors have the largest dot products with the vector for your current question. 

In [176]:
import numpy as np
from sentence_transformers import SentenceTransformer

# Your code here; load the sentence encoder (see https://www.sbert.net/ for documentation). A good choice of model is "all-MiniLM-L6-v2"
sentence_encoder = SentenceTransformer('all-MiniLM-L6-v2')

def compute_question_encodings(data):
    """
    For each example in the data, add a field called 'question_encoding' to the example, which is the vector encoding of the question.
    """
    # Your code here
    vectors = sentence_encoder.encode([d['question'] for d in data],show_progress_bar=False)
    for example, vector in zip(data, vectors):
        example['question_encoding'] = np.array(vector)


def select_similar_examples(question, few_shot_data, num_examples=4):
    """
    Return a list containing 4 of the elements of few_shot_data, selected with questions most semantically similar to the given question. 
    The most similar question should be the LAST element of the list, second most similar should be the second to last element, etc.
    The reason is that in the few-shot prompt, **you want the best example to be the most recent one.**

    To rank by semantic similarity, first compute the vector for the current question, then compute its dot product with
    all training set vectors (hint: you may want to vectorize this computation using numpy). Then sort by dot product.

    You should take advantage of the 'question_encoding' field that you added to each example in compute_question_encodings.
    """
    # Your code here
    # hint: when you call .encode with your sentence encoder, use show_progress_bar=False to avoid tons of printouts
    current_question_encoding = sentence_encoder.encode([question], show_progress_bar=False)[0]
    similarities = np.dot(current_question_encoding, np.array([example['question_encoding'] for example in few_shot_data]).T)
    sorted_indices = np.argsort(similarities)[::-1]
    selected_examples = [few_shot_data[i] for i in sorted_indices[:num_examples]]
    
    return selected_examples



In [177]:
# first precompute all the vectors for the training set
compute_question_encodings(splits['train'])

In [178]:
# this call will again take a few minutes, even if you batched; feel free to debug on smaller sets of dev
predictions = predict_greedy_fewshot(gpt2_model, splits['test'], splits['train'], example_selection_method=select_similar_examples)
print('4-shot prompting with similar examples, execution acc:', check_execution_accuracy(predictions, splits['test']))

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 625, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 544, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 599, but `max_length` is set to 512. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 513, but `max_length` is set to 512. This can lead to unexpected behavior. You should conside

4-shot prompting with similar examples, execution acc: 0.3285198555956679


You should now achieve about 34% accuracy. The autograder will check that you get >30%.

Save your predictions.

In [179]:
save_predictions(predictions, 'similar4shot_predictions.txt')

Once again, inspect some of your predictions (from prompting with similar examples) compared to the correct outputs, and describe some common types of errors in your report. Are there any differences compared to the finetuned model, or are the types of errors pretty similar? You can also check the exact match accuracy again.

In [183]:
print('4-shot prompting with similar examples, execution acc:', check_execution_accuracy(predictions, splits['test']))

4-shot prompting with similar examples, execution acc: 0.3285198555956679


In [186]:
finetuned_cnt,len(splits['test']),finetuned_cnt/len(splits['test'])

(58, 277, 0.20938628158844766)

In [ ]:
cnt=0
for i in range(len(finetuned_predictions)):
    if finetuned_predictions[i]!=splits['test'][i]['sql']:
        cnt+=1
        print(i,finetuned_predictions[i],"\n",splits['test'][i]['sql'])
print(cnt)

In [194]:

# assert len(predictions) == len(data)
# correct = 0
for p, d in zip(predictions, splits['test']):
    try:
        if query_db(p) != d['db_answer']:
            print(p,"\n",d)
    except: # failed to execute
        pass


 SELECT COUNT(*) FROM CITY AS CITYalias0 WHERE CITYalias0.POPULATION = ( SELECT MAX( CITYalias1.POPULATION ) FROM CITY AS CITYalias1 WHERE CITYalias1.STATE_NAME = "new mexico" ) AND CITYalias0.STATE_NAME = "new mexico" ; 
 {'question-split': 'test', 'question': 'where is the most populated area of new mexico', 'sql': 'SELECT CITYalias0.CITY_NAME FROM CITY AS CITYalias0 WHERE CITYalias0.POPULATION = ( SELECT MAX( CITYalias1.POPULATION ) FROM CITY AS CITYalias1 WHERE CITYalias1.STATE_NAME = "new mexico" ) AND CITYalias0.STATE_NAME = "new mexico" ;', 'db_answer': [('albuquerque',)]}
 SELECT CITYalias0.CITY_NAME FROM CITY AS CITYalias0 WHERE CITYalias0.POPULATION = ( SELECT MAX( CITYalias1.POPULATION ) FROM CITY AS CITYalias1 WHERE CITYalias1.STATE_NAME = "texas" ) AND CITYalias0.STATE_NAME = "texas" ; 
 {'question-split': 'test', 'question': 'how large is texas', 'sql': 'SELECT STATEalias0.AREA FROM STATE AS STATEalias0 WHERE STATEalias0.STATE_NAME = "texas" ;', 'db_answer': [(266807.0,)]

Finally, do some open-ended exploration to try to improve your performance on this dataset as much as possible (whether for finetuning or prompting). No hard requirement on how much to improve (or to improve at all), but please discuss the ideas you tried + how effective they were in your report. (Be careful with the GPU memory if you're using Kaggle, though- we're already nearly capping out the GPU memory in a few places with the current settings.)

A non-exhaustive list of possible ideas:
* Use a different similarity metric for selecting examples in few-shot prompting
* Use more examples in few-shot prompting
* Load a different base model than GPT2-Medium, or look into calling the OpenAI API
* Tune the hyperparameters used for finetuning
* Try to combine few-shot prompting with finetuning

##### Few shot prompting with cosine similarity

In [191]:
from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity_metric(question_vector, example_vectors):
    # Make sure question_vector and example_vectors are 2D arrays
    if len(question_vector.shape) == 1:
        question_vector = question_vector.reshape(1, -1)
    if len(example_vectors.shape) == 1:
        example_vectors = example_vectors.reshape(1, -1)
    
    similarities = cosine_similarity(question_vector, example_vectors)
    return similarities


def select_cosine_examples(question, few_shot_data, num_examples=4):
    current_question_encoding = sentence_encoder.encode([question], show_progress_bar=False)[0]
    current_question_encoding = current_question_encoding.reshape(1, -1)
    similarities = [cosine_similarity(current_question_encoding, example['question_encoding'].reshape(1, -1)) for example in few_shot_data]
    sorted_indices = np.argsort(similarities)[::-1]
    selected_examples = [few_shot_data[int(i)] for i in sorted_indices[:num_examples]]  # Cast indices to integers
    
    return selected_examples


In [192]:
predictions = predict_greedy_fewshot(gpt2_model, splits['test'], splits['train'], example_selection_method=select_cosine_examples)
print('4-shot prompting with cosine similar examples, execution acc:', check_execution_accuracy(predictions, splits['test']))

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


4-shot prompting with cosine similar examples, execution acc: 0.036101083032490974


##### Different parameters for fine tuning

In [164]:
from transformers import TrainingArguments, DataCollatorWithPadding

# Define the output directory to save checkpoints
output_dir = "/kaggle/working/checkpoint2"

gpt2_model = AutoModelForCausalLM.from_pretrained('gpt2-medium').to(device)
# Training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,  
    num_train_epochs=5,
    learning_rate=4e-5,
    per_device_train_batch_size=4,  
    gradient_accumulation_steps=8,
    warmup_steps=20, 
    weight_decay=0.05, 
    per_device_eval_batch_size=8,
    save_steps=80, 
    evaluation_strategy="steps",  
    eval_steps=80,  
    save_total_limit=2,  
    report_to="none", 
    fp16=True,   
    load_best_model_at_end=True,  
    logging_steps=80
)

# print(train_dataset.__getitem__(0))
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

trainer=Trainer(
    model=gpt2_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    eval_dataset=dev_dataset
)

trainer.train()
# model.save_pretrained(output_dir)

Step,Training Loss,Validation Loss
80,0.851700,0.024004


TrainOutput(global_step=85, training_loss=0.8030535874997868, metrics={'train_runtime': 378.6747, 'train_samples_per_second': 7.223, 'train_steps_per_second': 0.224, 'total_flos': 1487793023582208.0, 'train_loss': 0.8030535874997868, 'epoch': 4.96})

In [167]:
# !rm -r "/kaggle/working/checkpoint2/checkpoint-50"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [165]:
finetuned_model_2 = AutoModelForCausalLM.from_pretrained('/kaggle/working/checkpoint2/checkpoint-80')
finetuned_predictions = predict_greedy(finetuned_model_2, splits['test'])
print('finetuned execution acc:', check_execution_accuracy(finetuned_predictions, splits['test']))

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


finetuned execution acc: 0.33212996389891697


In [166]:
del finetuned_model_2
del trainer
import gc
gc.collect()
torch.cuda.empty_cache()

Your final submission should include the following files:

* hw4.ipynb (this file; please rename to match)
* finetuned_predictions.txt
* similar4shot_predictions.txt
* report.pdf